In [18]:
import pandas as pd

# Read data from census.csv into a DataFrame    
df = pd.read_csv('census.csv')

df.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
df.shape

(32561, 15)

In [7]:
import pandas_profiling

# Generate a profile report for the census DataFrame
profile = pandas_profiling.ProfileReport(df)
profile.to_file(output_file="census.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
df['age'].describe(percentiles=[.03, .05, .1, .2, .3, .4, .5, .6 , .7, .8, .9, .98, 1])

count    32561.000000
mean        38.581647
std         13.640433
min         17.000000
3%          19.000000
5%          19.000000
10%         22.000000
20%         26.000000
30%         30.000000
40%         33.000000
50%         37.000000
60%         41.000000
70%         45.000000
80%         50.000000
90%         58.000000
98%         69.000000
100%        90.000000
max         90.000000
Name: age, dtype: float64

Observations from pandas profiling
# There are 23 duplicate rows.  We need to remove them
# Vast majority of data is from the US.  Because of the small sample size of data for the remaining countries, it's better to exclude them and work on a US specific model
# 95% of the population is between the age of 19 and 69.  We will train and limit the model to predictions in that range
# Empty values have ?.  Will assess/address later
# High correlation between education-num and education.  Will use education-num since it's numerical value



In [27]:
import sys
print("\n".join(sys.path))

/Users/skotha/Projects/Learning/udacity/mlops/unit4/api_deployment/starter/data
/Users/skotha/miniconda3/envs/api-deployment/lib/python38.zip
/Users/skotha/miniconda3/envs/api-deployment/lib/python3.8
/Users/skotha/miniconda3/envs/api-deployment/lib/python3.8/lib-dynload

/Users/skotha/miniconda3/envs/api-deployment/lib/python3.8/site-packages


In [19]:
# Remove duplicates from the DataFrame
df = df.drop_duplicates()

# Filter data to only include 'United-States' in the native-country column
df = df[df['native-country'] == 'United-States']

# Filter age to be between 19 and 69
df = df[(df['age'] >= 19) & (df['age'] <= 69)]

df.describe()

,age,fnlgt,education-num,capital-gain,capital-loss,hours-per-week
count,27680.000000,2.768000e+04,27680.000000,27680.000000,27680.000000,27680.000000
mean,38.577529,1.873549e+05,10.270882,1108.706250,89.193064,41.273410
std,12.456327,1.049531e+05,2.345488,7458.749397,405.892849,11.801847
min,19.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.159845e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.770265e+05,10.000000,0.000000,0.000000,40.000000
75%,47.000000,2.342710e+05,13.000000,0.000000,0.000000,45.000000
max,69.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000
